#### Datasets :
1. Indian Liver Patient Records  - https://www.kaggle.com/uciml/indian-liver-patient-records

2. Productivity Prediction of Garment Employees Data Set - https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

#### **1. Indian Liver Patient Records** 
  - Prediction with KNN Algorithm
  - Prediction with Naive Bayes Algorithm

In [ ]:
ilp_dataset = pd.read_csv('https://raw.githubusercontent.com/shivshaktisahoo/test/main/indian_liver_patient.csv')
ilp_dataset.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [ ]:
ilp_dataset.info()
ilp_dataset.Albumin_and_Globulin_Ratio.describe()
ilp_dataset['Albumin_and_Globulin_Ratio'][240:256]
ilp_dataset['Albumin_and_Globulin_Ratio'] = ilp_dataset['Albumin_and_Globulin_Ratio'].fillna(round(ilp_dataset.Albumin_and_Globulin_Ratio.describe()['mean'], 2))
ilp_dataset['Gender'] = ilp_dataset['Gender'].map({"Male":1, "Female":0})
ilp_dataset = ilp_dataset.rename(columns={'Dataset':'LiverPatient'})
ilp_dataset['LiverPatient'] = ilp_dataset['LiverPatient'].map({1:1, 2:0})

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         583 non-null    int64  
 1   Gender                      583 non-null    object 
 2   Total_Bilirubin             583 non-null    float64
 3   Direct_Bilirubin            583 non-null    float64
 4   Alkaline_Phosphotase        583 non-null    int64  
 5   Alamine_Aminotransferase    583 non-null    int64  
 6   Aspartate_Aminotransferase  583 non-null    int64  
 7   Total_Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin_and_Globulin_Ratio  579 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB


In [ ]:
ilp_dataset

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,LiverPatient
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
578,60,1,0.5,0.1,500,20,34,5.9,1.6,0.37,0
579,40,1,0.6,0.1,98,35,31,6.0,3.2,1.10,1
580,52,1,0.8,0.2,245,48,49,6.4,3.2,1.00,1
581,31,1,1.3,0.5,184,29,32,6.8,3.4,1.00,1


In [ ]:
ilp_corr = ilp_dataset.corr()
ilp_corr

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,LiverPatient
Age,1.000000,0.056560,0.011763,0.007529,0.080425,-0.086883,-0.019910,-0.187461,-0.265924,-0.216109,0.137351
Gender,0.056560,1.000000,0.089291,0.100436,-0.027496,0.082332,0.080336,-0.089121,-0.093799,-0.003442,0.082416
Total_Bilirubin,0.011763,0.089291,1.000000,0.874618,0.206669,0.214065,0.237831,-0.008099,-0.222250,-0.206184,0.220208
Direct_Bilirubin,0.007529,0.100436,0.874618,1.000000,0.234939,0.233894,0.257544,-0.000139,-0.228531,-0.200030,0.246046
Alkaline_Phosphotase,0.080425,-0.027496,0.206669,0.234939,1.000000,0.125680,0.167196,-0.028514,-0.165453,-0.233989,0.184866
Alamine_Aminotransferase,-0.086883,0.082332,0.214065,0.233894,0.125680,1.000000,0.791966,-0.042518,-0.029742,-0.002395,0.163416
Aspartate_Aminotransferase,-0.019910,0.080336,0.237831,0.257544,0.167196,0.791966,1.000000,-0.025645,-0.085290,-0.070040,0.151934
Total_Protiens,-0.187461,-0.089121,-0.008099,-0.000139,-0.028514,-0.042518,-0.025645,1.000000,0.784053,0.233916,-0.035008
Albumin,-0.265924,-0.093799,-0.222250,-0.228531,-0.165453,-0.029742,-0.085290,0.784053,1.000000,0.686360,-0.161388
Albumin_and_Globulin_Ratio,-0.216109,-0.003442,-0.206184,-0.200030,-0.233989,-0.002395,-0.070040,0.233916,0.686360,1.000000,-0.162349


In [ ]:
# relationship between attribute and target with minimum threshold
MIN_THRES = 0.04         
final_attr = []
for i in ilp_corr.drop(['LiverPatient'],axis=1):
  if abs(ilp_corr[i]['LiverPatient']) >= MIN_THRES:
      final_attr.append(i)
final_attr

['Age',
 'Gender',
 'Total_Bilirubin',
 'Direct_Bilirubin',
 'Alkaline_Phosphotase',
 'Alamine_Aminotransferase',
 'Aspartate_Aminotransferase',
 'Albumin',
 'Albumin_and_Globulin_Ratio']

In [ ]:
ilp_corr = ilp_dataset[final_attr].corr()
ilp_corr

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Albumin,Albumin_and_Globulin_Ratio
Age,1.000000,0.056560,0.011763,0.007529,0.080425,-0.086883,-0.019910,-0.265924,-0.216109
Gender,0.056560,1.000000,0.089291,0.100436,-0.027496,0.082332,0.080336,-0.093799,-0.003442
Total_Bilirubin,0.011763,0.089291,1.000000,0.874618,0.206669,0.214065,0.237831,-0.222250,-0.206184
Direct_Bilirubin,0.007529,0.100436,0.874618,1.000000,0.234939,0.233894,0.257544,-0.228531,-0.200030
Alkaline_Phosphotase,0.080425,-0.027496,0.206669,0.234939,1.000000,0.125680,0.167196,-0.165453,-0.233989
Alamine_Aminotransferase,-0.086883,0.082332,0.214065,0.233894,0.125680,1.000000,0.791966,-0.029742,-0.002395
Aspartate_Aminotransferase,-0.019910,0.080336,0.237831,0.257544,0.167196,0.791966,1.000000,-0.085290,-0.070040
Albumin,-0.265924,-0.093799,-0.222250,-0.228531,-0.165453,-0.029742,-0.085290,1.000000,0.686360
Albumin_and_Globulin_Ratio,-0.216109,-0.003442,-0.206184,-0.200030,-0.233989,-0.002395,-0.070040,0.686360,1.000000


In [ ]:
ilp_corr["Age"]["Gender"]

0.0565602513226444

In [ ]:
# relationship between attribute and attribute with maximum threshold
MAX_THRES = 0.85
list1 = list(ilp_corr.columns)
max_final_attr = []
for i in enumerate(list1):
  for j in list1[i[0]+1:]:
    if abs(ilp_corr[i[1]][j]) >= MAX_THRES:
      print(abs(ilp_corr[i[1]][j]),i[1],j)
      max_final_attr.append(i[1])
max_final_attr

0.8746179301164149 Total_Bilirubin Direct_Bilirubin


['Total_Bilirubin']

In [ ]:
X1 = ilp_dataset.drop(['LiverPatient','Total_Bilirubin'], axis=1)
Y1 = ilp_dataset['LiverPatient']
neigh = KNeighborsClassifier(n_neighbors=20)
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X1,Y1,test_size=0.1)

In [ ]:
neigh.fit(x_train,y_train)
neigh.score(x_test,y_test)

0.6271186440677966

In [ ]:
print(neigh.predict_proba(np.array([65,0,0.1,187,16,18,6.8,3.3,0.90]).reshape(1,-1)))
print(neigh.predict_proba(np.array([34,1,0.6,67,21,9,8.8,6.3,0.6]).reshape(1,-1)))

[[0.6 0.4]]
[[0.3 0.7]]


- Naive bayes Algorithm


In [ ]:
from sklearn.naive_bayes import GaussianNB
neigh = GaussianNB()
neigh.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
prediction = neigh.predict_proba(np.array([65,0,0.1,187,16,18,6.8,3.3,0.90]).reshape(1,-1))[0]
predicted_target = np.argmax(prediction)
predicted_percentage = round(np.max(prediction)*100,2)
print(f"Their is {predicted_percentage}% chance of target {predicted_target}")

Their is 99.78% chance of target 0


#### **2. Productivity Prediction of Garment Employees Data Set**
- Prediction with KNN Algorithm
- Prediction with Naive Bayes Algorithm

In [ ]:
garments_dataset = pd.read_csv('https://raw.githubusercontent.com/shivshaktisahoo/test/main/garments_worker_productivity.csv')
garments_dataset

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,3/11/2015,Quarter2,finishing,Wednesday,10,0.75,2.90,NaN,960,0,0.0,0,0,8.0,0.628333
1193,3/11/2015,Quarter2,finishing,Wednesday,8,0.70,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1194,3/11/2015,Quarter2,finishing,Wednesday,7,0.65,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1195,3/11/2015,Quarter2,finishing,Wednesday,9,0.75,2.90,NaN,1800,0,0.0,0,0,15.0,0.505889


In [ ]:
## preprocessing /////////////////
garments_dataset['wip'] = garments_dataset['wip'].fillna(0)
garments_dataset['quarter'] = garments_dataset['quarter'].map({'Quarter1':1, 'Quarter2':2, 'Quarter3':3, 'Quarter4':4, 'Quarter5':5})
for i in range(len(garments_dataset)):
  garments_dataset['department'][i] = garments_dataset['department'][i].strip()
garments_dataset['department'] = garments_dataset['department'].map({'sweing':1, 'finishing':0})
garments_dataset['day'] = garments_dataset['day'].map({'Saturday':1, 'Sunday':2, 'Monday':3, 'Tuesday':4, 'Wednesday':5,'Thursday':6})
var1 = garments_dataset['actual_productivity']
for i in enumerate(var1):
  if var1[i[0]]>0 and var1[i[0]]<=0.4:
    var1[i[0]] = 1
  elif var1[i[0]]>0.4 and var1[i[0]]<=0.75:
    var1[i[0]] = 2
  else:
    var1[i[0]] = 3
garments_dataset

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,1,1,6,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,3.0
1,1/1/2015,1,0,6,1,0.75,3.94,0.0,960,0,0.0,0,0,8.0,3.0
2,1/1/2015,1,1,6,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,3.0
3,1/1/2015,1,1,6,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,3.0
4,1/1/2015,1,1,6,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,3/11/2015,2,0,5,10,0.75,2.90,0.0,960,0,0.0,0,0,8.0,2.0
1193,3/11/2015,2,0,5,8,0.70,3.90,0.0,960,0,0.0,0,0,8.0,2.0
1194,3/11/2015,2,0,5,7,0.65,3.90,0.0,960,0,0.0,0,0,8.0,2.0
1195,3/11/2015,2,0,5,9,0.75,2.90,0.0,1800,0,0.0,0,0,15.0,2.0


In [ ]:
g1 = garments_dataset.drop(['date'], axis=1)
garments_corr = g1.corr()
garments_corr

,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
quarter,1.000000,-0.012441,0.002283,0.021038,-0.112319,-0.000105,-0.064439,0.026654,-0.028237,-0.060779,-0.023884,0.189905,-0.005594,-0.074940
department,-0.012441,1.000000,0.018241,0.031753,-0.067508,0.874230,0.388442,0.677519,0.045782,0.049181,0.096701,0.300889,0.939360,0.017537
day,0.002283,0.018241,1.000000,-0.006399,-0.045685,0.010164,-0.022224,0.018933,-0.023396,-0.041165,0.001627,0.037138,0.022023,-0.017436
team,0.021038,0.031753,-0.006399,1.000000,0.030274,-0.110011,-0.011057,-0.096737,-0.007674,0.003796,0.026974,-0.011194,-0.075113,-0.107827
targeted_productivity,-0.112319,-0.067508,-0.045685,0.030274,1.000000,-0.069489,0.019035,-0.088557,0.032768,-0.056181,-0.053818,-0.209294,-0.084288,0.427344
smv,-0.000105,0.874230,0.010164,-0.110011,-0.069489,1.000000,0.322704,0.674887,0.032629,0.056863,0.105901,0.315388,0.912176,-0.040907
wip,-0.064439,0.388442,-0.022224,-0.011057,0.019035,0.322704,1.000000,0.276529,0.037946,-0.005101,-0.007119,0.053293,0.373908,0.065804
over_time,0.026654,0.677519,0.018933,-0.096737,-0.088557,0.674887,0.276529,1.000000,-0.004793,0.031038,-0.017913,0.059790,0.734164,0.008945
incentive,-0.028237,0.045782,-0.023396,-0.007674,0.032768,0.032629,0.037946,-0.004793,1.000000,-0.012024,-0.021140,-0.026607,0.049222,0.093027
idle_time,-0.060779,0.049181,-0.041165,0.003796,-0.056181,0.056863,-0.005101,0.031038,-0.012024,1.000000,0.559146,-0.011598,0.058049,-0.100667


In [ ]:
# relationship between attribute and targeted column with minimum threshold
MIN_THRES = 0.05
garments_final_attr = []
for i in garments_corr.drop(['actual_productivity'],axis=1):
  if abs(garments_corr[i]['actual_productivity']) >= MIN_THRES:
      garments_final_attr.append(i)
garments_final_attr

['quarter',
 'team',
 'targeted_productivity',
 'wip',
 'incentive',
 'idle_time',
 'idle_men',
 'no_of_style_change']

In [ ]:
# relationship between attribute and attribute with maximum threshold
MAX_THRES = 0.85
list1 = list(garments_corr.columns)
max_final_attr = []
for i in enumerate(list1):
  for j in list1[i[0]+1:]:
    if abs(garments_corr[i[1]][j]) >= MAX_THRES:
      print(abs(garments_corr[i[1]][j]),i[1],j)
      max_final_attr.append(i[1])
max_final_attr

0.8742295037807908 department smv
0.93935964980006 department no_of_workers
0.9121763124916773 smv no_of_workers


['department', 'department', 'smv']

In [ ]:
X = garments_dataset.drop(['date','actual_productivity','department', 'smv'], axis=1)
Y = garments_dataset['actual_productivity']
neigh = KNeighborsClassifier(n_neighbors=20)
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X,Y,test_size=0.1)

In [ ]:
X

,quarter,day,team,targeted_productivity,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers
0,1,6,8,0.80,1108.0,7080,98,0.0,0,0,59.0
1,1,6,1,0.75,0.0,960,0,0.0,0,0,8.0
2,1,6,11,0.80,968.0,3660,50,0.0,0,0,30.5
3,1,6,12,0.80,968.0,3660,50,0.0,0,0,30.5
4,1,6,6,0.80,1170.0,1920,50,0.0,0,0,56.0
...,...,...,...,...,...,...,...,...,...,...,...
1192,2,5,10,0.75,0.0,960,0,0.0,0,0,8.0
1193,2,5,8,0.70,0.0,960,0,0.0,0,0,8.0
1194,2,5,7,0.65,0.0,960,0,0.0,0,0,8.0
1195,2,5,9,0.75,0.0,1800,0,0.0,0,0,15.0


In [ ]:
neigh.fit(x_train,y_train)
neigh.score(x_test,y_test)

0.5583333333333333

In [ ]:
print(neigh.predict_proba(np.array([1,	6,	8,	0.80,	1108.0,	7080,	98,	0.0,	0,	0,	59.0]).reshape(1,-1)))
print(neigh.predict_proba(np.array([2,	5, 10,	0.75,	0.0,	960,	0,	0.0,	0,	0,	8.0]).reshape(1,-1)))

[[0.  0.1 0.9]]
[[0.05 0.6  0.35]]


In [ ]:
prediction = neigh.predict_proba(np.array([2,	5, 10,	0.75,	0.0,	960,	0,	0.0,	0,	0,	8.0]).reshape(1,-1))[0]
predicted_target = np.argmax(prediction)
predicted_percentage = round(np.max(prediction)*100,2)
print(f"Their is {predicted_percentage}% chance of target {predicted_target}")

Their is 60.0% chance of target 1


- Naive bayes Algorithm

In [ ]:
from sklearn.naive_bayes import GaussianNB
neigh = GaussianNB()
neigh.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
prediction = neigh.predict_proba(np.array([2,	5, 10,	0.75,	0.0,	960,	0,	0.0,	0,	0,	8.0]).reshape(1,-1))[0]
predicted_target = np.argmax(prediction)
predicted_percentage = round(np.max(prediction)*100,2)
print(f"Their is {predicted_percentage}% chance of target {predicted_target}")

Their is 99.71% chance of target 2
